In [1]:
import pandas

Let's create a dictionary for replacing the antibiotic 3 letter codes with their proper names; will be useful in Tables

In [2]:
drug_names_table = pandas.read_csv('dat/drugs/drug_names_lookup.csv')
drug_names_table.set_index('DRUG', inplace=True)
drug_names_lookup={}
for idx, row in drug_names_table.iterrows():
    drug_names_lookup[idx] = row.DRUG_NAME.capitalize()
drug_names_lookup

{'AMI': 'Amikacin',
 'BDQ': 'Bedaquiline',
 'CAP': 'Capreomycin',
 'CFZ': 'Clofazimine',
 'DLM': 'Delamanid',
 'EMB': 'Ethambutol',
 'ETH': 'Ethionamide',
 'INH': 'Isoniazid',
 'KAN': 'Kanamycin',
 'LEV': 'Levofloxacin',
 'LZD': 'Linezolid',
 'MXF': 'Moxifloxacin',
 'PZA': 'Pyrazinamide',
 'RFB': 'Rifabutin',
 'RIF': 'Rifampicin',
 'STM': 'Streptomycin'}

Load in the drugs in the WHOv2 catalogue, on the UKMYC plate, those we've chosen to use MGIT samples for etc

In [3]:
who_drugs = list(pandas.read_csv('dat/drugs/who2_drugs.csv').drug)
plate_drugs = list(pandas.read_csv('dat/drugs/plate_drugs.csv').drug)
other_drugs = list(pandas.read_csv('dat/drugs/other_drugs.csv').drug)
mgit_drugs = list(pandas.read_csv('dat/drugs/mgit_drugs.csv').drug)

plate_drug_qualities = [str(i)+"_QUALITY" for i in plate_drugs]

Now load in the 1,000 samples from the CRyPTIC project and the 1,663 MGIT samples for the five drugs where we either have relatively few resistant samples or aren't present on the UKMYC plate designs

In [4]:
UKMYC = pandas.read_csv('dat/UKMYC_1000_samples.csv')
UKMYC.set_index(['ENA_RUN_ACCESSION', 'UNIQUEID'], inplace=True)
UKMYC[:2]

,,ENA_STUDY_ACCESSION,ENA_SAMPLE_ACCESSION,HAS_ALL_DRUGS,N_SUSCEPTIBLE,N_RESISTANT,N_HIGH_QUALITY,AMI,BDQ,CFZ,DLM,...,DLM_QUALITY,EMB_QUALITY,ETH_QUALITY,INH_QUALITY,KAN_QUALITY,LEV_QUALITY,LZD_QUALITY,MXF_QUALITY,RIF_QUALITY,FASTQ_FTP
ENA_RUN_ACCESSION,UNIQUEID,,,,,,,,,,,,,,,,,,,,,
ERR4810791,site.02.subj.0068.lab.22A018.iso.1,PRJEB41194,SAMEA7542366,True,6,6,9,S,S,R,S,...,HIGH,HIGH,HIGH,HIGH,HIGH,LOW,LOW,HIGH,HIGH,ftp.sra.ebi.ac.uk/vol1/fastq/ERR481/001/ERR481...
ERR4810943,site.02.subj.0091.lab.22A043.iso.1,PRJEB41194,SAMEA7544003,True,5,7,11,S,S,S,S,...,HIGH,HIGH,HIGH,HIGH,HIGH,HIGH,LOW,HIGH,HIGH,ftp.sra.ebi.ac.uk/vol1/fastq/ERR481/003/ERR481...


In [5]:
MGIT = pandas.read_csv('dat/MGIT_1663_samples.csv')
MGIT.set_index(['ENA_RUN_ACCESSION','UNIQUEID'], inplace=True)
MGIT[:2]

,,WCHG_ID,ENA_SAMPLE_ACCESSION,ENA_STUDY_ACCESSION,HAS_ALL_DRUGS,N_SUSCEPTIBLE,N_RESISTANT,N_DRUGS,BDQ,CAP,LZD,PZA,STM,FASTQ_FTP
ENA_RUN_ACCESSION,UNIQUEID,,,,,,,,,,,,,
ERR9992611,site.10.subj.BC02023023.lab.BC02023023.iso.1,WTCHG_933274_70255001,SAMEA110419968,PRJEB55007,False,0,2,2,R,NaN,R,NaN,NaN,ftp.sra.ebi.ac.uk/vol1/fastq/ERR999/001/ERR999...
ERR13289349,site.10.subj.BH02818474.lab.BH02818474.iso.1,WTCHG_934880_70665042,SAMEA115747815,PRJEB76547,False,1,1,2,S,NaN,R,NaN,NaN,ftp.sra.ebi.ac.uk/vol1/fastq/ERR132/049/ERR132...


Create a table summarising the proportion of resistant samples by drug in this dataset

In [6]:
def find_position(df, drugs):
    current_position = {}
    for i in drugs:
        current_position[i] = []
        for phenotype in ['R', 'S']:
            current_position[i].append((df[i]==phenotype).sum())
    
    df = pandas.DataFrame.from_dict(current_position,orient='index',columns=['R','S'])
    df.sort_values(by='R',ascending=False, inplace=True)
    df['TOTAL'] = df['R'] + df['S']
    df['%R'] = (df['R'] / df['TOTAL'] * 100)
    return df

summary_ukmyc = find_position(UKMYC, plate_drugs)
summary_ukmyc['%R'] = summary_ukmyc['%R'].map('{:,.1f}'.format)
summary_ukmyc['Phenotype'] = 'UKMYC'
summary_ukmyc

,R,S,TOTAL,%R,Phenotype
INH,500,500,1000,50.0,UKMYC
RIF,476,524,1000,47.6,UKMYC
LEV,323,677,1000,32.3,UKMYC
KAN,289,711,1000,28.9,UKMYC
MXF,288,712,1000,28.8,UKMYC
AMI,288,712,1000,28.8,UKMYC
EMB,288,712,1000,28.8,UKMYC
ETH,288,712,1000,28.8,UKMYC
CFZ,288,712,1000,28.8,UKMYC
DLM,140,860,1000,14.0,UKMYC


Repeat for the MGIT samples (remembering that each sample has a varying number of MGIT results which we will describe shortly)

In [7]:
summary_mgit = find_position(MGIT, mgit_drugs)
summary_mgit['%R'] = summary_mgit['%R'].map('{:,.1f}'.format)
summary_mgit['Phenotype'] = 'MGIT'
summary_mgit

,R,S,TOTAL,%R,Phenotype
BDQ,394,380,774,50.9,MGIT
PZA,291,336,627,46.4,MGIT
CAP,256,575,831,30.8,MGIT
STM,252,362,614,41.0,MGIT
LZD,105,675,780,13.5,MGIT


In [8]:
summary = pandas.concat([summary_ukmyc, summary_mgit])
summary = summary[['Phenotype', '%R', 'R', 'S', 'TOTAL']]
summary

,Phenotype,%R,R,S,TOTAL
INH,UKMYC,50.0,500,500,1000
RIF,UKMYC,47.6,476,524,1000
LEV,UKMYC,32.3,323,677,1000
KAN,UKMYC,28.9,289,711,1000
MXF,UKMYC,28.8,288,712,1000
AMI,UKMYC,28.8,288,712,1000
EMB,UKMYC,28.8,288,712,1000
ETH,UKMYC,28.8,288,712,1000
CFZ,UKMYC,28.8,288,712,1000
DLM,UKMYC,14.0,140,860,1000


Finally, let's prepare this table for incorporation in the LaTeX manuscript

In [9]:
nice_summary = summary.rename(drug_names_lookup)
print(nice_summary.to_latex())

\begin{tabular}{lllrrr}
\toprule
 & Phenotype & %R & R & S & TOTAL \\
\midrule
Isoniazid & UKMYC & 50.0 & 500 & 500 & 1000 \\
Rifampicin & UKMYC & 47.6 & 476 & 524 & 1000 \\
Levofloxacin & UKMYC & 32.3 & 323 & 677 & 1000 \\
Kanamycin & UKMYC & 28.9 & 289 & 711 & 1000 \\
Moxifloxacin & UKMYC & 28.8 & 288 & 712 & 1000 \\
Amikacin & UKMYC & 28.8 & 288 & 712 & 1000 \\
Ethambutol & UKMYC & 28.8 & 288 & 712 & 1000 \\
Ethionamide & UKMYC & 28.8 & 288 & 712 & 1000 \\
Clofazimine & UKMYC & 28.8 & 288 & 712 & 1000 \\
Delamanid & UKMYC & 14.0 & 140 & 860 & 1000 \\
Linezolid & UKMYC & 11.7 & 117 & 883 & 1000 \\
Bedaquiline & UKMYC & 6.5 & 65 & 935 & 1000 \\
Bedaquiline & MGIT & 50.9 & 394 & 380 & 774 \\
Pyrazinamide & MGIT & 46.4 & 291 & 336 & 627 \\
Capreomycin & MGIT & 30.8 & 256 & 575 & 831 \\
Streptomycin & MGIT & 41.0 & 252 & 362 & 614 \\
Linezolid & MGIT & 13.5 & 105 & 675 & 780 \\
\bottomrule
\end{tabular}



To be able to join to the results from the genetics we need to move from a "short, wide" format to a "long, thin" format which will also let us combine both datasets into one. First the UKMYC data 

In [10]:
UKMYC.columns.name='DRUG'
plate_samples_phenotypes = UKMYC[plate_drugs].stack().to_frame(name='BINARY_PHENOTYPE')
plate_samples_qualities = UKMYC[plate_drug_qualities]
plate_samples_qualities.rename(columns={str(i)+"_QUALITY":str(i) for i in plate_drugs}, inplace=True)
plate_samples_qualities = plate_samples_qualities[plate_drugs].stack().to_frame(name='PHENOTYPE_QUALITY')
plate_samples_phenotypes = plate_samples_phenotypes.join(plate_samples_qualities)
plate_samples_phenotypes['PHENOTYPE_METHOD'] = 'UKMYC'
plate_samples_phenotypes[:3]

/var/folders/84/9nb2gsqj4hlcxztvwf19wg680000gq/T/ipykernel_18682/1620681778.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plate_samples_qualities.rename(columns={str(i)+"_QUALITY":str(i) for i in plate_drugs}, inplace=True)


BINARY_PHENOTYPE  \
ENA_RUN_ACCESSION UNIQUEID                           DRUG                    
ERR4810791        site.02.subj.0068.lab.22A018.iso.1 KAN                 S   
                                                     LEV                 R   
                                                     LZD                 R   

                                                          PHENOTYPE_QUALITY  \
ENA_RUN_ACCESSION UNIQUEID                           DRUG                     
ERR4810791        site.02.subj.0068.lab.22A018.iso.1 KAN               HIGH   
                                                     LEV                LOW   
                                                     LZD                LOW   

                                                          PHENOTYPE_METHOD  
ENA_RUN_ACCESSION UNIQUEID                           DRUG                   
ERR4810791        site.02.subj.0068.lab.22A018.iso.1 KAN             UKMYC  
                                                     LEV             UKMYC  
                                                     LZD             UKMYC

..and now the MGIT data

In [11]:
MGIT.columns.name='DRUG'
mgit_samples_phenotypes = MGIT[mgit_drugs].stack().to_frame(name='BINARY_PHENOTYPE')
mgit_samples_phenotypes['PHENOTYPE_QUALITY'] = None
mgit_samples_phenotypes['PHENOTYPE_METHOD'] = 'MGIT'
mgit_samples_phenotypes[:3]

BINARY_PHENOTYPE  \
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                    
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD                 R   
                                                               BDQ                 R   
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD                 R   

                                                                    PHENOTYPE_QUALITY  \
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                     
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD               None   
                                                               BDQ               None   
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD               None   

                                                                    PHENOTYPE_METHOD  
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                   
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD              MGIT  
                                                               BDQ              MGIT  
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD              MGIT

Before we go on, let's take a look at how many MGIT results each sample has

In [12]:
mgit_samples_summary = mgit_samples_phenotypes.reset_index()
mgit_samples_summary = mgit_samples_summary[['ENA_RUN_ACCESSION', 'DRUG']].groupby('ENA_RUN_ACCESSION').count() #.sort_values(by='ENA_RUN_ACCESSION', ascending=False)
mgit_samples_summary.rename(columns={'DRUG':'DRUG_COUNT'}, inplace=True)
mgit_samples_summary = mgit_samples_summary.DRUG_COUNT.value_counts().sort_index()
mgit_samples_summary = pandas.DataFrame(mgit_samples_summary)
mgit_samples_summary

,count
DRUG_COUNT,
1,347
2,702
3,581
4,33


In [13]:
print(mgit_samples_summary.to_latex())

\begin{tabular}{lr}
\toprule
 & count \\
DRUG_COUNT &  \\
\midrule
1 & 347 \\
2 & 702 \\
3 & 581 \\
4 & 33 \\
\bottomrule
\end{tabular}



Now we can concatenate these two datasets to make analysis easier later on

In [14]:
phenotypes = pandas.concat([mgit_samples_phenotypes, plate_samples_phenotypes])
phenotypes

BINARY_PHENOTYPE  \
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                    
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD                 R   
                                                               BDQ                 R   
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD                 R   
                                                               BDQ                 S   
ERR9992662        site.10.subj.IF02701910.lab.IF02701910.iso.1 LZD                 R   
...                                                                              ...   
ERR4812280        site.05.subj.CA-0634.lab.CO-12310-18.iso.1   DLM                 S   
                                                               RIF                 S   
                                                               BDQ                 S   
                                                               ETH                 S   
                                                               CFZ                 S   

                                                                    PHENOTYPE_QUALITY  \
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                     
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD               None   
                                                               BDQ               None   
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD               None   
                                                               BDQ               None   
ERR9992662        site.10.subj.IF02701910.lab.IF02701910.iso.1 LZD               None   
...                                                                               ...   
ERR4812280        site.05.subj.CA-0634.lab.CO-12310-18.iso.1   DLM               HIGH   
                                                               RIF               HIGH   
                                                               BDQ                LOW   
                                                               ETH               HIGH   
                                                               CFZ               HIGH   

                                                                    PHENOTYPE_METHOD  
ENA_RUN_ACCESSION UNIQUEID                                     DRUG                   
ERR9992611        site.10.subj.BC02023023.lab.BC02023023.iso.1 LZD              MGIT  
                                                               BDQ              MGIT  
ERR13289349       site.10.subj.BH02818474.lab.BH02818474.iso.1 LZD              MGIT  
                                                               BDQ              MGIT  
ERR9992662        site.10.subj.IF02701910.lab.IF02701910.iso.1 LZD              MGIT  
...                                                                              ...  
ERR4812280        site.05.subj.CA-0634.lab.CO-12310-18.iso.1   DLM             UKMYC  
                                                               RIF             UKMYC  
                                                               BDQ             UKMYC  
                                                               ETH             UKMYC  
                                                               CFZ             UKMYC  

[15626 rows x 3 columns]

Be paranoid and check there are no duplicated rows as a result of combining these two datasets

In [15]:
assert phenotypes.index.duplicated().sum()==0

Finally write the table to disc for later

In [16]:
phenotypes.to_csv('dat/PHENOTYPES.csv')